## Урок 3

In [1]:
from typing import List, Optional
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import scipy.stats as st
from scipy.stats import probplot, ks_2samp

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import missingno as msno
import xgboost as xgb

from sklearn.metrics import (roc_auc_score, roc_curve, auc, confusion_matrix, accuracy_score, \
                            classification_report, plot_confusion_matrix, plot_precision_recall_curve, \
                            precision_recall_curve, recall_score, plot_roc_curve)
import catboost as cb
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
# plt.style.use('fivethirtyeight')
# %config InlineBackend.figure_format = 'svg'
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv(r"./assignment_2_train.csv")
test = pd.read_csv(r"./assignment_2_test.csv")

In [3]:
train.head(2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
test.head(2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3287000,1,7415038,226.0,W,12473,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3287001,0,7415054,3072.0,W,15651,417.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [5]:
cat_cols = train.select_dtypes(include=[np.object]).columns.tolist()
cat_cols

['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']

In [6]:
for ii in train[cat_cols]:
    train[ii] = train[ii].astype(str)

In [7]:
cat_features_idx = np.where(train.drop('isFraud', axis=1).dtypes == "object")[0].tolist()
cat_features_idx

[3, 7, 9, 14, 15, 45, 46, 47, 48, 49, 50, 51, 52, 53]

In [8]:
for ii in train.drop('isFraud', axis=1).select_dtypes(include=['float', 'int']).columns.tolist():
    train[ii] = train[ii].fillna(-1)

In [9]:
X4 = train.drop(['isFraud'], axis=1)
y4 = train['isFraud']
X_train4, X_valid4, y_train4, y_valid4 = train_test_split(X4, y4, test_size=0.2, random_state=99, stratify=y4)

In [10]:
cb_params = {
    "n_estimators": 1500,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 50,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

In [15]:
model_cb4 = cb.CatBoostClassifier(**cb_params)
model_cb4.fit(X_train4, y_train4, eval_set=[(X_train4, y_train4), (X_valid4, y_valid4)], cat_features=cat_features_idx)

0:	test: 0.6215528	test1: 0.6194003	best: 0.6194003 (0)	total: 588ms	remaining: 14m 40s
25:	test: 0.8395464	test1: 0.8333882	best: 0.8333882 (25)	total: 11.4s	remaining: 10m 46s
50:	test: 0.8537770	test1: 0.8479772	best: 0.8479772 (50)	total: 22.1s	remaining: 10m 29s
75:	test: 0.8631775	test1: 0.8551333	best: 0.8551333 (75)	total: 32.9s	remaining: 10m 16s
100:	test: 0.8696869	test1: 0.8601079	best: 0.8601079 (100)	total: 43.6s	remaining: 10m 3s
125:	test: 0.8728865	test1: 0.8638313	best: 0.8638866 (123)	total: 54.2s	remaining: 9m 50s
150:	test: 0.8774091	test1: 0.8676607	best: 0.8676607 (150)	total: 1m 4s	remaining: 9m 38s
175:	test: 0.8819457	test1: 0.8720511	best: 0.8720511 (175)	total: 1m 15s	remaining: 9m 27s
200:	test: 0.8863090	test1: 0.8762741	best: 0.8762741 (200)	total: 1m 26s	remaining: 9m 15s
225:	test: 0.8897830	test1: 0.8795568	best: 0.8795568 (225)	total: 1m 36s	remaining: 9m 1s
250:	test: 0.8915717	test1: 0.8814177	best: 0.8814177 (250)	total: 1m 46s	remaining: 8m 49s
27

In [11]:
# Посмотрим какие результаты модель показывает на assignment_2_test(лидерборде)
for ii in test[cat_cols]:
    test[ii] = test[ii].astype(str)
    
for ii in test.drop('isFraud', axis=1).select_dtypes(include=['float', 'int']).columns.tolist():
    test[ii] = test[ii].fillna(-1)

In [12]:
cat_features_idxx = np.where(test.drop('isFraud', axis=1).dtypes == "object")[0].tolist()
cat_features_idxx

[3, 7, 9, 14, 15, 45, 46, 47, 48, 49, 50, 51, 52, 53]

In [13]:
X_test_catb = test.drop('isFraud', axis=1)
y_test_catb = test['isFraud']

In [19]:
model_cb4_pred_proba = model_cb4.predict_proba(X_test_catb)

In [20]:
roc_auc_score(y_test_catb, model_cb4_pred_proba[:,1])

0.8654995455074463

Выводы: Модель показала стабильный результат на обучающей и валидационной выборках(разница менее 2%), переобучение отсутствует, также модель показала хороший результат на лидерборде с разницей менее 5% от показателеля на валидационной выборке. Такие хорошие показатели обусловлены синтетическим происхождением данных и соответствием распределением статистик выборок. 

### Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [14]:
X4.shape

(180000, 393)

In [15]:
X_train, X_part, y_train, y_part = train_test_split(X4, y4, train_size=0.6, random_state=99, stratify=y4)

In [16]:
X_valid, X_test, y_valid, y_test = train_test_split(X_part, y_part, train_size=0.5, random_state=99, stratify=y_part)

In [17]:
X_train.shape, X_valid.shape, X_test.shape

((108000, 393), (36000, 393), (36000, 393))

In [18]:
# Соотношение классов
y_train[y_train==1].count() / y_train[y_train==0].count(), \
y_valid[y_valid==1].count() / y_valid[y_valid==0].count(), \
y_test[y_test==1].count() / y_test[y_test==0].count()

(0.02940475623123481, 0.029394944527050212, 0.029394944527050212)

In [21]:
model_cb5 = cb.CatBoostClassifier(**cb_params)
model_cb5.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], cat_features=cat_features_idx)

0:	test: 0.6789397	test1: 0.6878085	best: 0.6878085 (0)	total: 480ms	remaining: 12m
50:	test: 0.8499162	test1: 0.8469826	best: 0.8476461 (47)	total: 18.6s	remaining: 8m 48s
100:	test: 0.8660818	test1: 0.8647403	best: 0.8647403 (100)	total: 36s	remaining: 8m 18s
150:	test: 0.8748416	test1: 0.8720256	best: 0.8720256 (150)	total: 53s	remaining: 7m 53s
200:	test: 0.8839868	test1: 0.8796879	best: 0.8796879 (200)	total: 1m 10s	remaining: 7m 32s
250:	test: 0.8900731	test1: 0.8851295	best: 0.8851295 (250)	total: 1m 26s	remaining: 7m 12s
300:	test: 0.8946869	test1: 0.8888934	best: 0.8888934 (300)	total: 1m 43s	remaining: 6m 52s
350:	test: 0.8980165	test1: 0.8917620	best: 0.8917620 (350)	total: 2m	remaining: 6m 33s
400:	test: 0.8994873	test1: 0.8928102	best: 0.8928102 (400)	total: 2m 16s	remaining: 6m 14s
450:	test: 0.9006590	test1: 0.8935748	best: 0.8935748 (450)	total: 2m 32s	remaining: 5m 55s
500:	test: 0.9014083	test1: 0.8940899	best: 0.8941136 (496)	total: 2m 48s	remaining: 5m 36s
550:	test

In [66]:
print(f'train_auc: {round(roc_auc_score(y_train, model_cb5.predict_proba(X_train)[:,1]), 4)}')
print(f'valid_auc: {round(roc_auc_score(y_valid, model_cb5.predict_proba(X_valid)[:,1]), 4)}')
print(f'test_auc: {round(roc_auc_score(y_test, model_cb5.predict_proba(X_test)[:,1]), 4)}')
print(f'LB_auc: {round(roc_auc_score(y_test_catb, model_cb5.predict_proba(X_test_catb)[:,1]), 4)}')

train_auc: 0.9115
valid_auc: 0.902
test_auc: 0.9
LB_auc: 0.8609


Показатели аналогичны предыдущим тестам, разница между тестом и лидербордом осталась на том же уровне.

### Задание 3: построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

In [31]:
from typing import List, Tuple
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [35]:
pred = model_cb5.predict_proba(X_test)[:,1]

In [46]:
np.random.seed(99)
scores1 = create_bootstrap_metrics(y_test, pred, roc_auc_score)
calculate_confidence_interval(scores1, conf_interval=0.95)

(0.8884487671314546, 0.9109710135320566)

In [47]:
np.random.seed(99)
scores2 = create_bootstrap_metrics(y_valid, model_cb5.predict_proba(X_valid)[:,1], roc_auc_score)
calculate_confidence_interval(scores2, conf_interval=0.95)

(0.8907482740199953, 0.9132809502901694)

Результат на лидерборде(LB_auc 0.8609) лежит вблизи слева доверительного интервала, но не внутри, однако в то же время нельзя сделать вывод, что модель явно неустойчива.

### Задание 4: выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

In [55]:
X_adv = pd.concat([
    train.drop('isFraud', axis=1), test.drop('isFraud', axis=1)
])
y_adv = np.hstack(
    (np.ones(train.shape[0]), np.zeros(test.shape[0])))

X_adv = X_adv.reset_index(drop=True)
assert X_adv.shape[0] == y_adv.shape[0], 'неверная размерность'

In [68]:
X_adv

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,86400,68.500,W,13926,-1.0,150.0,discover,142.0,credit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,2987001,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,credit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2987002,86469,59.000,W,4663,490.0,150.0,visa,166.0,debit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,2987003,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,debit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2987004,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279996,3386996,10091528,368.990,W,13964,496.0,150.0,mastercard,224.0,debit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
279997,3386997,10091533,445.330,W,10616,583.0,150.0,visa,226.0,credit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
279998,3386998,10091544,15.226,C,9803,583.0,150.0,visa,226.0,credit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
279999,3386999,10091549,34.742,C,16062,500.0,185.0,mastercard,137.0,credit,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [73]:
model_cb6 = cb.CatBoostClassifier(**cb_params)
model_cb6.fit(X_adv, y_adv, cat_features=cat_features_idx)

y_pred_adv = model_cb6.predict_proba(X_adv)
score = roc_auc_score(y_adv, y_pred_adv[:, 1])
print(round(score, 4))

0:	total: 613ms	remaining: 15m 18s
50:	total: 30.9s	remaining: 14m 36s
100:	total: 58.2s	remaining: 13m 25s
150:	total: 1m 22s	remaining: 12m 19s
200:	total: 1m 47s	remaining: 11m 32s
250:	total: 2m 11s	remaining: 10m 53s
300:	total: 2m 35s	remaining: 10m 19s
350:	total: 2m 59s	remaining: 9m 47s
400:	total: 3m 22s	remaining: 9m 15s
450:	total: 3m 46s	remaining: 8m 47s
500:	total: 4m 11s	remaining: 8m 20s
550:	total: 4m 35s	remaining: 7m 54s
600:	total: 4m 59s	remaining: 7m 27s
650:	total: 5m 22s	remaining: 7m 1s
700:	total: 5m 46s	remaining: 6m 35s
750:	total: 6m 10s	remaining: 6m 9s
800:	total: 6m 34s	remaining: 5m 44s
850:	total: 6m 58s	remaining: 5m 19s
900:	total: 7m 23s	remaining: 4m 55s
950:	total: 7m 50s	remaining: 4m 31s
1000:	total: 8m 17s	remaining: 4m 7s
1050:	total: 8m 43s	remaining: 3m 43s
1100:	total: 9m 10s	remaining: 3m 19s
1150:	total: 9m 37s	remaining: 2m 54s
1200:	total: 10m 3s	remaining: 2m 30s
1250:	total: 10m 30s	remaining: 2m 5s
1300:	total: 10m 56s	remaining: 1m

Выводы: гипотеза о том, что распределения на трейне и на тесте совпадают, неверна(CV-score: 0.9992), распределения на трейне и на тесте сильно различаются. 

### Задание 5: сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

In [82]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
  
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict_proba(x_train)
        y_valid_pred = estimator.predict_proba(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred[:,1]))
        fold_valid_scores.append(metric(y_valid, y_valid_pred[:,1]))
        oof_predictions[valid_idx] = y_valid_pred[:,1]

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")
    
    print(estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions)
    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [83]:
model_cb7 = cb.CatBoostClassifier(**cb_params, cat_features=cat_features_idx)

In [84]:
%%time
cv_strategy = KFold(n_splits=5)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    train.drop('isFraud', axis=1), train['isFraud'], model_cb7, metric=roc_auc_score, cv_strategy=cv_strategy
)

0:	total: 398ms	remaining: 9m 57s
50:	total: 16.2s	remaining: 7m 39s
100:	total: 31.6s	remaining: 7m 18s
150:	total: 47.4s	remaining: 7m 3s
200:	total: 1m 3s	remaining: 6m 48s
250:	total: 1m 18s	remaining: 6m 32s
300:	total: 1m 34s	remaining: 6m 15s
350:	total: 1m 49s	remaining: 5m 59s
400:	total: 2m 5s	remaining: 5m 42s
450:	total: 2m 20s	remaining: 5m 26s
500:	total: 2m 36s	remaining: 5m 11s
550:	total: 2m 51s	remaining: 4m 55s
600:	total: 3m 6s	remaining: 4m 39s
650:	total: 3m 22s	remaining: 4m 23s
700:	total: 3m 37s	remaining: 4m 7s
750:	total: 3m 52s	remaining: 3m 51s
800:	total: 4m 7s	remaining: 3m 36s
850:	total: 4m 23s	remaining: 3m 21s
900:	total: 4m 39s	remaining: 3m 5s
950:	total: 4m 54s	remaining: 2m 50s
1000:	total: 5m 10s	remaining: 2m 34s
1050:	total: 5m 26s	remaining: 2m 19s
1100:	total: 5m 41s	remaining: 2m 3s
1150:	total: 5m 57s	remaining: 1m 48s
1200:	total: 6m 12s	remaining: 1m 32s
1250:	total: 6m 27s	remaining: 1m 17s
1300:	total: 6m 41s	remaining: 1m 1s
1350:	tota

In [86]:
model_cb7_pred_proba = model_cb7.predict_proba(X_test_catb)
roc_auc_score(y_test_catb, model_cb7_pred_proba[:,1])

0.8695781716971943

Выводы: Кросс-валидация устойчивая на всех фолдах, дисперсия низкая. 